In [1]:
import numpy as np

import sys
sys.path.insert(
    1,
    '/home/trduong/Data/interpretable_machine_learning/Source Code/my_work/lib'
)

import numpy as np
import pandas as pd
import logging
import incremental_ps_score_estimator as ipse
import utils


from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from geneticalgorithm import geneticalgorithm as ga


In [2]:
file_path = "https://msalicedatapublic.blob.core.windows.net/datasets/Pricing/pricing_sample.csv"
train_data = pd.read_csv(file_path)

In [3]:
train_data['treatment'] = np.where(train_data['price'] == 1, 1, 0)
train_data['price'] = np.where(train_data['price'] == 1, 1, 0.85)

In [4]:
outcome = "demand"
treatment = "treatment"
col = list(train_data.columns)
col.remove("price")
print(col)

cov = col[:]
cov.remove(treatment)
cov.remove(outcome)
cov.remove('income')
print(cov)

features = col[:]
features.remove(outcome)

print(features)

['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased', 'income', 'demand', 'treatment']
['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased']
['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased', 'income', 'treatment']


In [5]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(train_data, test_size=0.1)

In [6]:
## Fit treatment
model_t = LogisticRegression()
model_t.fit(train_data[cov], train_data[treatment])

train_data['p1'] = model_t.predict_proba(train_data[cov])[:,1]
train_data['p0'] = 1 - train_data['p1']

train_data["prediction"] = np.where(train_data["p1"] >= 0.5, 1, 0)

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 1000)
model_y.fit(train_data[features], train_data[outcome])

    

GradientBoostingRegressor(n_estimators=1000, random_state=0)

In [7]:
train_data.shape

(9000, 15)

## Optimization

In [8]:
def gamma_fn(X):
    return -3 - 14 * (X["income"] < 1)

def beta_fn(X):
    return 20 + 0.5 * (X["avg_hours"]) + 5 * (X["days_visited"] > 4)

def demand_fn(data, T):
    Y = gamma_fn(data) * T + beta_fn(data)
    return Y

def true_te(x, n, stats):
    if x < 1:
        subdata = df[df["income"] < 1].sample(n=n, replace=True)
    else:
        subdata = df[df["income"] >= 1].sample(n=n, replace=True)
    te_array = subdata["price"] * gamma_fn(subdata) / (subdata["demand"])
    if stats == "mean":
        return np.mean(te_array)
    elif stats == "median":
        return np.median(te_array)
    elif isinstance(stats, int):
        return np.percentile(te_array, stats)
    
def revenue_fn(data, discount_level1, discount_level2, baseline_T, policy):
    policy_price = baseline_T * (1 - discount_level1) * policy + baseline_T * (1 - discount_level2) * (1 - policy)
    demand = demand_fn(data, policy_price)
    rev = demand * policy_price
    return rev

def fitness_function(propensity_score, solution_idx):
    policy = np.where(propensity_score >= 0.5, 1, 0)
    return np.mean(revenue_fn(train_data,  0., 0.15, 1, policy))


In [9]:
train_data.head()

,account_age,age,avg_hours,days_visited,friends_count,has_membership,is_US,songs_purchased,income,price,demand,treatment,p1,p0,prediction
5466,2,28,8.508083,6,9,1,0,6.093560,1.878202,0.85,26.554042,0,0.446247,0.553753,0
9435,1,54,9.578934,4,14,0,1,5.795057,1.480526,1.00,21.789467,1,0.417497,0.582503,0
8443,3,33,4.086286,7,8,0,1,4.088569,1.020977,1.00,24.043143,1,0.464931,0.535069,0
1483,4,54,3.762321,6,9,1,0,7.755960,0.370185,0.85,13.281160,0,0.428216,0.571784,0
9227,3,20,3.852820,4,6,1,1,3.956176,1.761313,1.00,18.926410,1,0.453593,0.546407,0


In [10]:
#0: discount, level 2, 1: no discount - level 1
np.mean(revenue_fn(train_data,  0.0, 0.15, 1, train_data.treatment.values))

14.308830571002158

In [11]:
np.mean(train_data['demand']*train_data['price'])

14.377760848779936

## PyGrad

In [ ]:
import pygad
import numpy

function_inputs = np.random.rand(train_data.shape[0]) # Function inputs.

num_generations = 2000 # Number of generations.
num_parents_mating = 500 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 1500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -1
init_range_high = 1

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "two_points" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 20 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]

print("Running model")
# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()


Running model
Generation = 1
Fitness    = 13.710571892440957
Change     = 13.710571892440957


Generation = 2
Fitness    = 13.721454545184855
Change     = 0.010882652743898547




In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

In [ ]:
proposed_policy = np.where(solution >= 0.5, 1, 0)

In [ ]:
proposed_policy

In [ ]:
discount_all = np.mean(revenue_fn(train_data,  0.0, 0.15, 1, np.zeros(train_data.shape[0])))
no_discount = np.mean(revenue_fn(train_data,  0.0, 0.15, 1, np.ones(train_data.shape[0])))

In [ ]:
print("Current policy {:.4f}".format(discount_all))
print("Discount all {:.4f}".format(discount_all))
print("No discount {:.4f}".format(no_discount))
print("-"*100)
print("Proposed strategy {:.4f}".format(solution_fitness))

In [ ]:
from collections import Counter 
print(Counter(proposed_policy))
print(Counter(train_data.treatment.values))


In [ ]:
current_policy = train_data.treatment.values

In [ ]:
proposed_policy = [str(x) for x in proposed_policy]
current_policy = [str(x) for x in current_policy]

In [ ]:
14.5488 / 13.3610 * 100